In [ ]:
from pavlov import *
monitor(-2)

In [ ]:
from pavlov import stats
stats.plot(['*wry-sights*', '*prize-remedy*', '*hairy-surge*', -2, -1], 'elo-mohex', 'μ', skip=1, head=240, fill=True)

In [ ]:
logs.paths(-2)